In [2]:
import requests
import base64
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.svm import SVC

CLIENT_ID = CLIENT_ID_GOES_HERE
CLIENT_SECRET = CLIENT_SECRET_GOES_HERE
CLIENT_CREDS = f"{CLIENT_ID}:{CLIENT_SECRET}"
ENCODED = base64.b64encode(CLIENT_CREDS.encode())

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
#access_token = auth_response_data['access_token']
BASE_URL = 'https://api.spotify.com/v1/'

### index?
#r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
#                 headers=headers, 
#                 params={'include_groups': 'album', 'limit': 50})

In [3]:
### Authentication
access_token = ACCESS_TOKEN_GOES_HERE
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [4]:
### Fetching track data

liked_request = requests.get(BASE_URL + "me/tracks", headers=headers).json()
liked_tracks = pd.DataFrame(liked_request)
liked_tracks = pd.json_normalize(liked_tracks['items'])

fnf_id = PLAYLIST_ID_GOES_HERE
fnf_request = requests.get(BASE_URL + "playlists/" + fnf_id, headers=headers).json()
fnf_tracks = pd.json_normalize(fnf_request)
fnf_tracks = pd.json_normalize(fnf_tracks["tracks.items"]).T
fnf_tracks = pd.json_normalize(fnf_tracks[0])

# you can find the URIs inside the Spotify app by right-clicking

In [5]:
def clean_data(df_name):
    short_tracks = df_name[['track.id', 'track.name', 'track.popularity']]
    short_tracks = short_tracks.rename(columns={"track.id": "id", "track.name": "name", "track.popularity": "popularity"})
    short_tracks["url"] = BASE_URL + "tracks/" + short_tracks["id"]
    short_tracks["features_url"] = BASE_URL + "audio-features/" + short_tracks["id"]
    return short_tracks

In [6]:
def add_features(df_name):
    features_df = pd.DataFrame()
    for i in range(len(df_name)):
        track_features = requests.get(df_name["features_url"].iloc[i], headers=headers).json()
        track_info = pd.DataFrame([track_features])
        features_df = pd.concat([features_df, track_info])
    full_tracks = pd.merge(df_name, features_df, on='id', how='left')
    return full_tracks

In [7]:
liked_tracks = clean_data(liked_tracks)
liked_tracks = add_features(liked_tracks)
liked_tracks["fav"] = 1

fnf_tracks = clean_data(fnf_tracks)
fnf_tracks = add_features(fnf_tracks)
fnf_tracks["fav"] = 0

all_tracks = pd.concat([liked_tracks, fnf_tracks])

In [8]:
test_tracks = all_tracks[["danceability", "energy", "loudness", "acousticness", "instrumentalness", "liveness", "speechiness", "tempo", "fav"]]
x_train, x_test, y_train, y_test = train_test_split(test_tracks.drop('fav', axis=1), test_tracks.fav ,test_size = 0.2, random_state=13)

x_train = StandardScaler(with_mean=0, with_std=1).fit_transform(x_train)
x_test = StandardScaler(with_mean=0, with_std=1).fit_transform(x_test)

clf = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)
predictions = clf.predict(x_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.83      0.95      0.88        20
           1       0.00      0.00      0.00         4

    accuracy                           0.79        24
   macro avg       0.41      0.47      0.44        24
weighted avg       0.69      0.79      0.74        24



In [9]:
new_id = NEW_PLAYLIST_ID_GOES_HERE
new_request = requests.get(BASE_URL + "playlists/" + fnf_id, headers=headers).json()
new_tracks = pd.json_normalize(new_request)
new_tracks = pd.json_normalize(new_tracks["tracks.items"]).T
new_tracks = pd.json_normalize(new_tracks[0])

new_tracks = clean_data(new_tracks)
new_tracks = add_features(new_tracks)


In [10]:
new_tracks = new_tracks[["danceability", "energy", "loudness", "acousticness", "instrumentalness", "liveness", "speechiness", "tempo"]]
display(clf.predict(new_tracks))

C:\Users\17087\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)